In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.display.max_rows = None
pd.options.display.max_columns = None
from sklearn.cross_validation import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
#import lightgbm as lgb 
import math as mt
from sklearn.ensemble import GradientBoostingClassifier
import catboost as cb
import featuretools as ft

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
header_df = pd.read_csv('header.csv')
train_df = pd.read_csv('train.csv',header=None)
test_df = pd.read_csv('test.csv',header=None)
sample_sub = pd.read_csv('sample_submission.csv')

In [4]:
train_df.columns = header_df.columns 
col_list = []

for item in header_df.columns.values :
    if item != 'label' :
           col_list.append(item) 
            
test_df.columns = col_list
test_df.head()

,key,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54
0,258918,2352,293,23,256,45,201,151,228,215,799,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,258134,2790,88,8,330,35,1400,232,227,128,726,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,165265,2826,115,7,162,24,1020,232,233,134,2044,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,522141,3234,137,16,150,20,968,243,232,112,2536,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,569662,2810,58,22,124,25,3293,230,188,80,1624,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
cols = ['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10']
for col in cols :
    train_df.loc[:,col] = (train_df[col]-train_df[col].mean())/train_df[col].std()
    test_df.loc[:,col] = (test_df[col]-test_df[col].mean())/test_df[col].std()

In [5]:
train_df.head()

,key,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,label
0,447095,1.760236,-0.400681,1.056243,0.767480,2.868109,0.849059,1.490779,-0.675209,-1.792757,1.008586,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,113427,0.477016,-0.543572,-0.681745,-0.685560,-0.676802,1.129631,0.817983,0.185496,-0.485917,0.717096,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,66435,-1.460324,-0.847216,0.387786,-0.845441,-0.711053,-1.041277,0.705851,-1.080247,-1.165474,-0.928385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,8957,-0.055573,-0.186344,-1.483893,0.753373,-0.574051,-0.309741,0.444208,0.742423,0.167502,-0.590076,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,434631,0.251827,1.528352,2.660540,0.033907,0.932964,-0.242481,-3.443056,-2.143471,1.239110,-0.217030,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [6]:
train_df.describe()

,key,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,label
count,406709.000000,4.067090e+05,4.067090e+05,4.067090e+05,4.067090e+05,4.067090e+05,4.067090e+05,4.067090e+05,4.067090e+05,4.067090e+05,4.067090e+05,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000,406709.000000
mean,290590.177164,-2.014696e-16,-2.037125e-16,-4.419174e-17,1.557784e-15,-5.074325e-16,-2.754761e-16,-2.190828e-16,1.227140e-16,7.460359e-17,-4.315006e-17,0.448726,0.051405,0.436317,0.063552,0.005217,0.012963,0.008257,0.021241,0.002737,0.011379,0.000179,0.000275,0.002021,0.056224,0.021485,0.051585,0.029997,0.001006,0.000005,0.004940,0.005901,0.003329,0.006830,0.015834,0.001421,0.057572,0.099686,0.036599,0.000806,0.004413,0.001815,0.001640,0.198036,0.051902,0.044309,0.089951,0.077975,0.002756,0.003204,0.000221,0.000479,0.026953,0.023712,0.015144,0.487712
std,167743.065591,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.497365,0.220823,0.495928,0.243953,0.072044,0.113113,0.090490,0.144188,0.052241,0.106065,0.013396,0.016592,0.044911,0.230355,0.144993,0.221188,0.170579,0.031696,0.002218,0.070109,0.076591,0.057603,0.082364,0.124835,0.037672,0.232932,0.299581,0.187775,0.028387,0.066287,0.042559,0.040464,0.398520,0.221829,0.205782,0.286112,0.268132,0.052428,0.056511,0.014874,0.021891,0.161946,0.152151,0.122124,0.499850
min,1.000000,-3.933828e+00,-1.391990e+00,-1.884967e+00,-1.268657e+00,-3.759334e+00,-1.506335e+00,-7.928361e+00,-1.130745e+01,-3.726879e+00,-1.495506e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,145203.000000,-5.381212e-01,-8.740085e-01,-6.817445e-01,-7.607981e-01,-6.768024e-01,-7.978590e-01,-5.276082e-01,-5.233201e-01,-6.166013e-01,-7.222279e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,290625.000000,1.302969e-01,-2.577895e-01,-1.469790e-01,-2.435346e-01,-2.829234e-01,-2.283879e-01,2.199426e-01,1.348663e-01,1.068153e-02,-2.041920e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,435909.000000,7.272264e-01,9.389255e-01,5.214778e-01,5.652774e-01,3.849584e-01,6.261390e-01,7.058506e-01,6.917932e-01,6.641012e-01,4.301376e-01,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.0000

In [6]:
train_df.drop(['V50','V23'],axis=1,inplace=True)
test_df.drop(['V50','V23'],axis=1,inplace=True)

In [7]:
train_x, dev_x, train_y, dev_y = train_test_split(train_df.drop(['key','label'],axis=1), train_df['label'], test_size=0.33, random_state=42)


In [ ]:

#gives baseline scroe
cat_features_index = [i for i in range(10,50)]
clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=31, \
                            depth=10, iterations= 500, l2_leaf_reg= 10, learning_rate= 0.2)
clf.fit(train_x,train_y, cat_features= cat_features_index)
pred = clf.predict(dev_x)
print roc_auc_score(dev_y,pred)
print roc_auc_score(train_y,clf.predict(train_x))

0:	total: 932ms	remaining: 7m 45s
1:	total: 1.48s	remaining: 6m 8s
2:	total: 2.26s	remaining: 6m 14s
3:	total: 2.88s	remaining: 5m 57s
4:	total: 3.96s	remaining: 6m 32s
5:	total: 4.67s	remaining: 6m 24s
6:	total: 5.18s	remaining: 6m 5s
7:	total: 5.81s	remaining: 5m 57s
8:	total: 6.45s	remaining: 5m 51s
9:	total: 7.04s	remaining: 5m 45s
10:	total: 7.7s	remaining: 5m 42s
11:	total: 8.23s	remaining: 5m 34s
12:	total: 8.78s	remaining: 5m 28s
13:	total: 9.31s	remaining: 5m 23s
14:	total: 9.84s	remaining: 5m 18s
15:	total: 10.4s	remaining: 5m 13s
16:	total: 10.9s	remaining: 5m 10s
17:	total: 11.4s	remaining: 5m 5s
18:	total: 11.9s	remaining: 5m 2s
19:	total: 12.5s	remaining: 4m 59s
20:	total: 13s	remaining: 4m 56s
21:	total: 13.6s	remaining: 4m 54s
22:	total: 14.1s	remaining: 4m 52s
23:	total: 14.6s	remaining: 4m 50s
24:	total: 15.2s	remaining: 4m 48s
25:	total: 15.8s	remaining: 4m 47s
26:	total: 16.3s	remaining: 4m 46s
27:	total: 16.9s	remaining: 4m 44s
28:	total: 17.8s	remaining: 4m 49s
29

229:	total: 2m 36s	remaining: 3m 3s
230:	total: 2m 37s	remaining: 3m 3s
231:	total: 2m 38s	remaining: 3m 2s
232:	total: 2m 38s	remaining: 3m 2s
233:	total: 2m 39s	remaining: 3m 1s
234:	total: 2m 40s	remaining: 3m
235:	total: 2m 40s	remaining: 2m 59s
236:	total: 2m 41s	remaining: 2m 59s
237:	total: 2m 41s	remaining: 2m 58s
238:	total: 2m 42s	remaining: 2m 57s
239:	total: 2m 43s	remaining: 2m 56s
240:	total: 2m 43s	remaining: 2m 55s
241:	total: 2m 44s	remaining: 2m 55s
242:	total: 2m 45s	remaining: 2m 54s
243:	total: 2m 45s	remaining: 2m 53s
244:	total: 2m 46s	remaining: 2m 53s
245:	total: 2m 46s	remaining: 2m 52s
246:	total: 2m 47s	remaining: 2m 51s
247:	total: 2m 48s	remaining: 2m 50s
248:	total: 2m 48s	remaining: 2m 50s
249:	total: 2m 49s	remaining: 2m 49s
250:	total: 2m 50s	remaining: 2m 48s
251:	total: 2m 51s	remaining: 2m 48s
252:	total: 2m 51s	remaining: 2m 47s
253:	total: 2m 52s	remaining: 2m 46s
254:	total: 2m 52s	remaining: 2m 46s
255:	total: 2m 53s	remaining: 2m 45s
256:	total

455:	total: 4m 55s	remaining: 28.5s
456:	total: 4m 56s	remaining: 27.9s
457:	total: 4m 56s	remaining: 27.2s
458:	total: 4m 57s	remaining: 26.6s
459:	total: 4m 57s	remaining: 25.9s
460:	total: 4m 58s	remaining: 25.2s
461:	total: 4m 58s	remaining: 24.6s
462:	total: 4m 59s	remaining: 23.9s
463:	total: 5m	remaining: 23.3s
464:	total: 5m	remaining: 22.6s
465:	total: 5m 1s	remaining: 22s
466:	total: 5m 1s	remaining: 21.3s
467:	total: 5m 2s	remaining: 20.7s
468:	total: 5m 2s	remaining: 20s
469:	total: 5m 3s	remaining: 19.4s
470:	total: 5m 3s	remaining: 18.7s
471:	total: 5m 4s	remaining: 18s
472:	total: 5m 4s	remaining: 17.4s
473:	total: 5m 5s	remaining: 16.8s
474:	total: 5m 5s	remaining: 16.1s
475:	total: 5m 6s	remaining: 15.5s
476:	total: 5m 7s	remaining: 14.8s
477:	total: 5m 7s	remaining: 14.2s
478:	total: 5m 8s	remaining: 13.5s
479:	total: 5m 8s	remaining: 12.9s
480:	total: 5m 9s	remaining: 12.2s
481:	total: 5m 9s	remaining: 11.6s
482:	total: 5m 10s	remaining: 10.9s
483:	total: 5m 10s	rema

In [ ]:
cat_features_index = [i for i in range(10,50)]
interactions = []

col = ['V43','V11', 'V47', 'V46','V13','V37', 'V38', 'V45','V44','V27']
#cols = ['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10']
for feature_A in col :
    for feature_B in col :
          if feature_A > feature_B :
            print feature_A
            print feature_B
            train_x['interaction'] = train_x[feature_A] + train_x[feature_B]
            dev_x['interaction'] = dev_x[feature_A] + dev_x[feature_B]
            clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=31, \
                            depth=10, iterations= 500, l2_leaf_reg= 10, learning_rate= 0.2)
            clf.fit(train_x,train_y, cat_features= cat_features_index)
            y_pred = clf.predict(dev_x)
            score = roc_auc_score(dev_y,y_pred)
            #print score
            if score > 0.95 :
                print interactions
                interactions.append((feature_A, feature_B, round(score,3)))

print 'Top 10 interactions: %s' % sorted(interactions, key=lambda(x):x[2],reverse=True)[:10]

V43
V11
0:	total: 737ms	remaining: 6m 7s
1:	total: 1.28s	remaining: 5m 18s
2:	total: 1.96s	remaining: 5m 24s
3:	total: 2.83s	remaining: 5m 51s
4:	total: 3.42s	remaining: 5m 38s
5:	total: 4.13s	remaining: 5m 40s
6:	total: 4.87s	remaining: 5m 43s
7:	total: 5.58s	remaining: 5m 43s
8:	total: 6.3s	remaining: 5m 43s
9:	total: 7.16s	remaining: 5m 50s
10:	total: 7.8s	remaining: 5m 46s
11:	total: 8.91s	remaining: 6m 2s
12:	total: 9.83s	remaining: 6m 8s
13:	total: 11.6s	remaining: 6m 42s
14:	total: 12.5s	remaining: 6m 44s
15:	total: 13.2s	remaining: 6m 39s
16:	total: 13.8s	remaining: 6m 32s
17:	total: 14.4s	remaining: 6m 26s
18:	total: 15s	remaining: 6m 19s
19:	total: 15.9s	remaining: 6m 20s
20:	total: 16.7s	remaining: 6m 21s
21:	total: 17.5s	remaining: 6m 20s
22:	total: 18.6s	remaining: 6m 26s
23:	total: 19.3s	remaining: 6m 23s
24:	total: 19.9s	remaining: 6m 18s
25:	total: 20.5s	remaining: 6m 14s
26:	total: 21.2s	remaining: 6m 12s
27:	total: 22.1s	remaining: 6m 11s
28:	total: 22.7s	remaining: 6

229:	total: 2m 59s	remaining: 3m 30s
230:	total: 2m 59s	remaining: 3m 29s
231:	total: 3m	remaining: 3m 28s
232:	total: 3m 1s	remaining: 3m 27s
233:	total: 3m 1s	remaining: 3m 26s
234:	total: 3m 2s	remaining: 3m 25s
235:	total: 3m 3s	remaining: 3m 24s
236:	total: 3m 4s	remaining: 3m 24s
237:	total: 3m 5s	remaining: 3m 23s
238:	total: 3m 5s	remaining: 3m 23s
239:	total: 3m 7s	remaining: 3m 22s
240:	total: 3m 7s	remaining: 3m 21s
241:	total: 3m 8s	remaining: 3m 21s
242:	total: 3m 9s	remaining: 3m 20s
243:	total: 3m 9s	remaining: 3m 19s
244:	total: 3m 10s	remaining: 3m 18s
245:	total: 3m 11s	remaining: 3m 17s
246:	total: 3m 12s	remaining: 3m 16s
247:	total: 3m 12s	remaining: 3m 15s
248:	total: 3m 13s	remaining: 3m 15s
249:	total: 3m 14s	remaining: 3m 14s
250:	total: 3m 15s	remaining: 3m 13s
251:	total: 3m 15s	remaining: 3m 12s
252:	total: 3m 16s	remaining: 3m 11s
253:	total: 3m 17s	remaining: 3m 11s
254:	total: 3m 18s	remaining: 3m 10s
255:	total: 3m 19s	remaining: 3m 9s
256:	total: 3m 20s

455:	total: 5m 50s	remaining: 33.8s
456:	total: 5m 51s	remaining: 33.1s
457:	total: 5m 52s	remaining: 32.3s
458:	total: 5m 52s	remaining: 31.5s
459:	total: 5m 53s	remaining: 30.7s
460:	total: 5m 54s	remaining: 30s
461:	total: 5m 54s	remaining: 29.2s
462:	total: 5m 55s	remaining: 28.4s
463:	total: 5m 56s	remaining: 27.6s
464:	total: 5m 56s	remaining: 26.8s
465:	total: 5m 57s	remaining: 26.1s
466:	total: 5m 58s	remaining: 25.3s
467:	total: 5m 59s	remaining: 24.6s
468:	total: 6m	remaining: 23.8s
469:	total: 6m 1s	remaining: 23.1s
470:	total: 6m 2s	remaining: 22.3s
471:	total: 6m 3s	remaining: 21.6s
472:	total: 6m 4s	remaining: 20.8s
473:	total: 6m 5s	remaining: 20.1s
474:	total: 6m 6s	remaining: 19.3s
475:	total: 6m 7s	remaining: 18.5s
476:	total: 6m 7s	remaining: 17.7s
477:	total: 6m 8s	remaining: 17s
478:	total: 6m 9s	remaining: 16.2s
479:	total: 6m 10s	remaining: 15.4s
480:	total: 6m 11s	remaining: 14.7s
481:	total: 6m 12s	remaining: 13.9s
482:	total: 6m 13s	remaining: 13.1s
483:	total

185:	total: 2m 46s	remaining: 4m 41s
186:	total: 2m 47s	remaining: 4m 40s
187:	total: 2m 48s	remaining: 4m 39s
188:	total: 2m 49s	remaining: 4m 38s
189:	total: 2m 49s	remaining: 4m 37s
190:	total: 2m 50s	remaining: 4m 36s
191:	total: 2m 51s	remaining: 4m 35s
192:	total: 2m 52s	remaining: 4m 34s
193:	total: 2m 53s	remaining: 4m 33s
194:	total: 2m 53s	remaining: 4m 32s
195:	total: 2m 54s	remaining: 4m 31s
196:	total: 2m 56s	remaining: 4m 31s
197:	total: 2m 57s	remaining: 4m 31s
198:	total: 2m 59s	remaining: 4m 31s
199:	total: 3m	remaining: 4m 31s
200:	total: 3m 1s	remaining: 4m 30s
201:	total: 3m 2s	remaining: 4m 29s
202:	total: 3m 3s	remaining: 4m 28s
203:	total: 3m 4s	remaining: 4m 27s
204:	total: 3m 5s	remaining: 4m 26s
205:	total: 3m 6s	remaining: 4m 25s
206:	total: 3m 7s	remaining: 4m 24s
207:	total: 3m 8s	remaining: 4m 23s
208:	total: 3m 8s	remaining: 4m 22s
209:	total: 3m 9s	remaining: 4m 21s
210:	total: 3m 10s	remaining: 4m 20s
211:	total: 3m 11s	remaining: 4m 19s
212:	total: 3m 

409:	total: 6m 1s	remaining: 1m 19s
410:	total: 6m 1s	remaining: 1m 18s
411:	total: 6m 2s	remaining: 1m 17s
412:	total: 6m 3s	remaining: 1m 16s
413:	total: 6m 4s	remaining: 1m 15s
414:	total: 6m 5s	remaining: 1m 14s
415:	total: 6m 6s	remaining: 1m 13s
416:	total: 6m 7s	remaining: 1m 13s
417:	total: 6m 8s	remaining: 1m 12s
418:	total: 6m 8s	remaining: 1m 11s
419:	total: 6m 9s	remaining: 1m 10s
420:	total: 6m 10s	remaining: 1m 9s
421:	total: 6m 11s	remaining: 1m 8s
422:	total: 6m 12s	remaining: 1m 7s
423:	total: 6m 13s	remaining: 1m 6s
424:	total: 6m 14s	remaining: 1m 6s
425:	total: 6m 14s	remaining: 1m 5s
426:	total: 6m 15s	remaining: 1m 4s
427:	total: 6m 16s	remaining: 1m 3s
428:	total: 6m 17s	remaining: 1m 2s
429:	total: 6m 18s	remaining: 1m 1s
430:	total: 6m 18s	remaining: 1m
431:	total: 6m 19s	remaining: 59.8s
432:	total: 6m 20s	remaining: 58.9s
433:	total: 6m 21s	remaining: 58s
434:	total: 6m 22s	remaining: 57.1s
435:	total: 6m 23s	remaining: 56.2s
436:	total: 6m 23s	remaining: 55.

...

256:	total: 1m 45s	remaining: 1m 39s
257:	total: 1m 46s	remaining: 1m 39s
258:	total: 1m 46s	remaining: 1m 39s
259:	total: 1m 46s	remaining: 1m 38s
260:	total: 1m 47s	remaining: 1m 38s
261:	total: 1m 47s	remaining: 1m 37s
262:	total: 1m 48s	remaining: 1m 37s
263:	total: 1m 48s	remaining: 1m 37s
264:	total: 1m 49s	remaining: 1m 36s
265:	total: 1m 49s	remaining: 1m 36s
266:	total: 1m 49s	remaining: 1m 35s
267:	total: 1m 50s	remaining: 1m 35s
268:	total: 1m 50s	remaining: 1m 35s
269:	total: 1m 51s	remaining: 1m 34s
270:	total: 1m 51s	remaining: 1m 34s
271:	total: 1m 51s	remaining: 1m 33s
272:	total: 1m 52s	remaining: 1m 33s
273:	total: 1m 52s	remaining: 1m 33s
274:	total: 1m 53s	remaining: 1m 32s
275:	total: 1m 53s	remaining: 1m 32s
276:	total: 1m 53s	remaining: 1m 31s
277:	total: 1m 54s	remaining: 1m 31s
278:	total: 1m 54s	remaining: 1m 30s
279:	total: 1m 55s	remaining: 1m 30s
280:	total: 1m 55s	remaining: 1m 30s
281:	total: 1m 56s	remaining: 1m 29s
282:	total: 1m 56s	remaining: 1m 29s
2

485:	total: 3m 20s	remaining: 5.76s
486:	total: 3m 20s	remaining: 5.35s
487:	total: 3m 20s	remaining: 4.94s
488:	total: 3m 21s	remaining: 4.53s
489:	total: 3m 21s	remaining: 4.12s
490:	total: 3m 22s	remaining: 3.71s
491:	total: 3m 22s	remaining: 3.29s
492:	total: 3m 23s	remaining: 2.88s
493:	total: 3m 23s	remaining: 2.47s
494:	total: 3m 23s	remaining: 2.06s
495:	total: 3m 24s	remaining: 1.65s
496:	total: 3m 24s	remaining: 1.24s
497:	total: 3m 25s	remaining: 824ms
498:	total: 3m 25s	remaining: 412ms
499:	total: 3m 26s	remaining: 0us
V46
V13
0:	total: 359ms	remaining: 2m 59s
1:	total: 713ms	remaining: 2m 57s
2:	total: 1.06s	remaining: 2m 55s
3:	total: 1.43s	remaining: 2m 56s
4:	total: 1.79s	remaining: 2m 57s
5:	total: 2.16s	remaining: 2m 57s
6:	total: 2.52s	remaining: 2m 57s
7:	total: 2.92s	remaining: 2m 59s
8:	total: 3.3s	remaining: 3m
9:	total: 3.7s	remaining: 3m 1s
10:	total: 4.11s	remaining: 3m 2s
11:	total: 4.53s	remaining: 3m 4s
12:	total: 4.96s	remaining: 3m 5s
13:	total: 5.38s	re

218:	total: 1m 30s	remaining: 1m 55s
219:	total: 1m 30s	remaining: 1m 55s
220:	total: 1m 31s	remaining: 1m 54s
221:	total: 1m 31s	remaining: 1m 54s
222:	total: 1m 31s	remaining: 1m 54s
223:	total: 1m 32s	remaining: 1m 53s
224:	total: 1m 32s	remaining: 1m 53s
225:	total: 1m 33s	remaining: 1m 52s
226:	total: 1m 33s	remaining: 1m 52s
227:	total: 1m 33s	remaining: 1m 52s
228:	total: 1m 34s	remaining: 1m 51s
229:	total: 1m 34s	remaining: 1m 51s
230:	total: 1m 35s	remaining: 1m 50s
231:	total: 1m 35s	remaining: 1m 50s
232:	total: 1m 35s	remaining: 1m 49s
233:	total: 1m 36s	remaining: 1m 49s
234:	total: 1m 36s	remaining: 1m 49s
235:	total: 1m 37s	remaining: 1m 48s
236:	total: 1m 37s	remaining: 1m 48s
237:	total: 1m 37s	remaining: 1m 47s
238:	total: 1m 38s	remaining: 1m 47s
239:	total: 1m 38s	remaining: 1m 47s
240:	total: 1m 39s	remaining: 1m 46s
241:	total: 1m 39s	remaining: 1m 46s
242:	total: 1m 40s	remaining: 1m 45s
243:	total: 1m 40s	remaining: 1m 45s
244:	total: 1m 40s	remaining: 1m 44s
2

445:	total: 3m 4s	remaining: 22.3s
446:	total: 3m 4s	remaining: 21.9s
447:	total: 3m 5s	remaining: 21.5s
448:	total: 3m 6s	remaining: 21.1s
449:	total: 3m 6s	remaining: 20.7s
450:	total: 3m 7s	remaining: 20.3s
451:	total: 3m 7s	remaining: 19.9s
452:	total: 3m 7s	remaining: 19.5s
453:	total: 3m 8s	remaining: 19.1s
454:	total: 3m 8s	remaining: 18.7s
455:	total: 3m 9s	remaining: 18.3s
456:	total: 3m 10s	remaining: 17.9s
457:	total: 3m 11s	remaining: 17.5s
458:	total: 3m 11s	remaining: 17.1s
459:	total: 3m 12s	remaining: 16.8s
460:	total: 3m 13s	remaining: 16.4s
461:	total: 3m 15s	remaining: 16.1s
462:	total: 3m 15s	remaining: 15.6s
463:	total: 3m 16s	remaining: 15.2s
464:	total: 3m 16s	remaining: 14.8s
465:	total: 3m 17s	remaining: 14.4s
466:	total: 3m 17s	remaining: 14s
467:	total: 3m 17s	remaining: 13.5s
468:	total: 3m 18s	remaining: 13.1s
469:	total: 3m 18s	remaining: 12.7s
470:	total: 3m 19s	remaining: 12.3s
471:	total: 3m 19s	remaining: 11.9s
472:	total: 3m 20s	remaining: 11.4s
473:	

179:	total: 1m 12s	remaining: 2m 9s
180:	total: 1m 13s	remaining: 2m 8s
181:	total: 1m 13s	remaining: 2m 8s
182:	total: 1m 13s	remaining: 2m 8s
183:	total: 1m 14s	remaining: 2m 7s
184:	total: 1m 14s	remaining: 2m 7s
185:	total: 1m 15s	remaining: 2m 6s
186:	total: 1m 15s	remaining: 2m 6s
187:	total: 1m 15s	remaining: 2m 6s
188:	total: 1m 16s	remaining: 2m 5s
189:	total: 1m 16s	remaining: 2m 5s
190:	total: 1m 17s	remaining: 2m 4s
191:	total: 1m 17s	remaining: 2m 4s
192:	total: 1m 17s	remaining: 2m 4s
193:	total: 1m 18s	remaining: 2m 3s
194:	total: 1m 18s	remaining: 2m 3s
195:	total: 1m 19s	remaining: 2m 2s
196:	total: 1m 19s	remaining: 2m 2s
197:	total: 1m 20s	remaining: 2m 2s
198:	total: 1m 20s	remaining: 2m 1s
199:	total: 1m 20s	remaining: 2m 1s
200:	total: 1m 21s	remaining: 2m
201:	total: 1m 21s	remaining: 2m
202:	total: 1m 21s	remaining: 1m 59s
203:	total: 1m 22s	remaining: 1m 59s
204:	total: 1m 22s	remaining: 1m 59s
205:	total: 1m 23s	remaining: 1m 58s
206:	total: 1m 23s	remaining: 